<a href="https://colab.research.google.com/github/rohananpothle/Basic_ChatBot_Data_Science_Project/blob/main/Power_Bi_Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# powerbi_chatbot.py

import pandas as pd
import gradio as gr
import pickle

# ----------------------------- Helper Modules -----------------------------

def data_cleaning_recommendations(df):
    recommendations = []
    if df.isnull().values.any():
        recommendations.append("- Missing values found. Recommend filling with mean/median or removing rows.")
    if df.duplicated().any():
        recommendations.append("- Duplicate rows detected. Consider removing them.")
    for col in df.columns:
        if df[col].dtype == 'object':
            if df[col].str.contains(r'[^a-zA-Z0-9\s]').any():
                recommendations.append(f"- Column '{col}' contains special characters. Clean up may be required.")
        if 'date' in col.lower() or 'time' in col.lower():
            try:
                pd.to_datetime(df[col])
            except Exception:
                recommendations.append(f"- Column '{col}' has date/time format issues. Convert to datetime.")
    return "\n".join(recommendations) if recommendations else "No major cleaning needed."


def suggest_conditional_columns(df):
    suggestions = []
    lower_cols = [col.lower() for col in df.columns]
    if 'sales' in lower_cols and 'target' in lower_cols:
        suggestions.append("- Add column 'Performance': IF Sales > Target THEN 'Achieved' ELSE 'Pending'")
    if any('date' in col for col in lower_cols):
        suggestions.append("- Add 'Month', 'Quarter', 'Year' columns from date column for time-based analysis.")
    return "\n".join(suggestions) if suggestions else "No conditional columns detected."


def generate_power_query_steps(df):
    steps = []
    for col in df.columns:
        dtype = str(df[col].dtype)
        if dtype == 'object':
            steps.append(f"= Table.TransformColumnTypes(Source, {{\"{col}\", type text}})")
        elif 'int' in dtype or 'float' in dtype:
            steps.append(f"= Table.TransformColumnTypes(Source, {{\"{col}\", type number}})")
        elif 'datetime' in dtype:
            steps.append(f"= Table.TransformColumnTypes(Source, {{\"{col}\", type datetime}})")
    return "\n".join(steps)


def recommend_kpis_and_dax(df):
    dax_suggestions = []
    columns = [col.lower() for col in df.columns]

    metrics = {
        "revenue": [
            ("YoY Growth", "YoY Growth =\nVAR CurrentYear = SUM('Table'[Revenue])\nVAR LastYear = CALCULATE(SUM('Table'[Revenue]), SAMEPERIODLASTYEAR('Table'[Date]))\nRETURN (CurrentYear - LastYear) / LastYear"),
            ("MTD Revenue", "MTD Revenue = TOTALMTD(SUM('Table'[Revenue]), 'Table'[Date])")
        ],
        "sales": [
            ("Total Sales", "Total Sales = SUM('Table'[Sales])"),
            ("Average Sales", "Average Sales = AVERAGE('Table'[Sales])")
        ],
        "quantity": [
            ("Total Quantity", "Total Quantity = SUM('Table'[Quantity])")
        ],
        "profit": [
            ("Profit Margin", "Profit Margin = DIVIDE(SUM('Table'[Profit]), SUM('Table'[Revenue]))")
        ],
        "customerid": [
            ("Customer Count", "Customer Count = DISTINCTCOUNT('Table'[CustomerID])")
        ]
    }

    found_kpi = False
    for key, formulas in metrics.items():
        if key in columns:
            found_kpi = True
            for title, formula in formulas:
                dax_suggestions.append(f"Suggested KPI: {title}")
                dax_suggestions.append(formula)

    if not found_kpi:
        numeric_cols = df.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            dax_suggestions.append(f"Suggested KPI: Sum of {col}")
            dax_suggestions.append(f"{col}_Total = SUM('Table'[{col}])")

    return "\n\n".join(dax_suggestions)


def powerbi_project_steps():
    return """1. Import Data
2. Clean and transform data in Power Query
3. Create Date Table
4. Define relationships between tables
5. Create conditional columns and calculated columns
6. Create KPIs using DAX
7. Build report visuals (charts, tables, KPIs)
8. Add slicers, filters, tooltips
9. Test interactions and performance
10. Publish to Power BI Service
11. Set up auto refresh & row-level security if needed"""

# ----------------------------- Gradio UI -----------------------------

def chatbot_interface(file):
    df = pd.read_csv(file.name) if file.name.endswith(".csv") else pd.read_excel(file.name)

    cleaning = data_cleaning_recommendations(df)
    conditional = suggest_conditional_columns(df)
    power_query = generate_power_query_steps(df)
    dax = recommend_kpis_and_dax(df)
    project_steps = powerbi_project_steps()

    with open("uploaded_dataset.pkl", "wb") as f:
        pickle.dump(df, f)

    return cleaning, conditional, power_query, dax, project_steps

interface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.File(label="Upload Dataset (.csv or .xlsx)"),
    outputs=[
        gr.Textbox(label="Data Cleaning Suggestions"),
        gr.Textbox(label="Conditional Columns to Add"),
        gr.Textbox(label="Power Query Steps (M Code)"),
        gr.Textbox(label="KPI Recommendations + DAX"),
        gr.Textbox(label="Power BI Project Workflow")
    ],
    title="Power BI Chatbot Assistant",
    description="Upload your dataset to receive end-to-end Power BI recommendations: cleaning, Power Query steps, KPIs with DAX, and project steps."
)

if __name__ == "__main__":
    interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ece1587b8523286983.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
